# ALTEGRAD Kaggle challenge

# I) Imports

In [2]:
# !nvidia-smi

In [1]:
# Go back to root folder
%cd ..
%pwd

/users/eleves-b/2020/thomas.li/MVA/ALTEGRAD/kaggle_challenge


'/users/eleves-b/2020/thomas.li/MVA/ALTEGRAD/kaggle_challenge'

In [2]:
# Install packages
# !pip install wandb onnx -Uq

# Imports
import os
import time
import numpy as np
import matplotlib.pyplot as pyplot
import seaborn as sns

sns.set_theme()

import pandas as pd
from tqdm import tqdm
import torch
from torch import optim
import torch.nn as nn
from torch_geometric.data import DataLoader
from torch.utils.data import DataLoader as TorchDataLoader
from transformers import AutoTokenizer

# Imports from custom package
from src.dataloader import GraphTextDataset, GraphDataset, TextDataset

# from original_src.Model import Model

# from original_src.Model2 import Model

# Device configuration
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# WandB Login and Setup
import wandb

wandb.login()
PROJECT_NAME = "ALTEGRAD_kaggle"  # TODO

/users/eleves-b/2020/thomas.li/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda:0


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: qwertyr2. Use `wandb login --relogin` to force relogin


### default_config

In [6]:
# Default config as dictionary
finetune_config = dict(
    model_path="models/baseline_negative_sampling_contrastive_loss_v2_01282244/baseline_negative_sampling_contrastive_loss_v2_01282244_49.pth",
    model_name="distilbert-base-uncased",
    batch_size=32,
    graph_learning_rate=(1e-4) / 5,
    text_learning_rate=(3e-5) / 5,
    learning_rate=1e-4,
    epoch_equivalents=20,
    criterion="negative_sampling_contrastive_loss",
    similarity_fn="cosine_similarity",
)


# Default config as class instance
class Config_Class:
    def __init__(self, dictionary):
        for k, v in dictionary.items():
            setattr(self, k, v)


finetune_config_instance = Config_Class(finetune_config)

#### Debugging

Vizulize text

In [28]:
model, train_loader, val_loader, criterion, optimizer = make(
    finetune_config_instance, shuffle=False
)

/users/eleves-b/2020/thomas.li/.local/lib/python3.9/site-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)
/users/eleves-b/2020/thomas.li/.local/lib/python3.9/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 20 worker processes in total. Our suggested max number of worker in current system is 16, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


device : cuda


In [29]:
model.load_state_dict(torch.load("models/baseline_finetune/baseline_finetune.pth"))

<All keys matched successfully>

In [30]:
from sklearn.metrics.pairwise import cosine_similarity

train_similarity_matrix = evaluate(
    model,
    train_loader,
    criterion,
    cosine_similarity,
    LRAP_accuracy,
    "none",
    example_ct=1565,
    wandb_log=False,
)

 20%|██        | 169/826 [00:18<01:38,  6.66it/s]/users/eleves-b/2020/thomas.li/.local/lib/python3.9/site-packages/torch_geometric/data/collate.py:177: UserWarning: An output with one or more elements was resized since it had shape [1960], which does not match the required output shape [2, 1960]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at ../aten/src/ATen/native/Resize.cpp:28.)
  value = torch.cat(values, dim=cat_dim or 0, out=out)
 36%|███▌      | 298/826 [00:33<00:43, 12.27it/s]/users/eleves-b/2020/thomas.li/.local/lib/python3.9/site-packages/torch_geometric/data/collate.py:177: UserWarning: An output with one or more elements was resized since it had shape [2248], which does not match the required output shape [2, 2248]. This behavior is deprecated, and in a future PyTorch release 

In [55]:
batch_sampler, stats_dico = get_batch_sampler(train_similarity_matrix, batch_size=32)

{'len_miss_ranked': 8323, 'avg_miss_ranked': 2.8925868076414756, 'max_miss_ranked': 88, 'total_miss_ranked_pairs': 32398}


In [34]:
batch_sampler_sizes = []
for sublist in batch_sampler:
    batch_sampler_sizes.append(len(sublist))
batch_sampler_sizes = np.array(batch_sampler_sizes)

In [35]:
batch_sampler_sizes.max()

32

In [57]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
gt = np.load("./data/token_embedding_dict.npy", allow_pickle=True)[()]
train_dataset = GraphTextDataset(
    root="./data/", gt=gt, split="train", tokenizer=tokenizer
)
# finetune_train_loader = DataLoader(
#     train_dataset, batch_sampler=batch_sampler, num_workers=20
# )


finetune_train_loader = DataLoader(
    train_dataset, batch_sampler=batch_sampler, num_workers=20
)

/users/eleves-b/2020/thomas.li/.local/lib/python3.9/site-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)
/users/eleves-b/2020/thomas.li/.local/lib/python3.9/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 20 worker processes in total. Our suggested max number of worker in current system is 16, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [51]:
n_total_samples = 26000
batch_size = 32
regular_batch_sampler = np.random.permutation(n_total_samples)
regular_batch_sampler = regular_batch_sampler[
    : n_total_samples - (n_total_samples % batch_size)
]
regular_batch_sampler = regular_batch_sampler.reshape(
    n_total_samples // batch_size, batch_size
).tolist()

In [96]:
def get_empty_edge_index(batch):
    n_nodes = batch.x.shape[0]
    self_edge_index = torch.arange(n_nodes).unsqueeze(0)
    self_edge_index = torch.cat((self_edge_index, self_edge_index), dim=0)
    return self_edge_index


example_ct = 0
for batch_idx, batch in tqdm(enumerate(finetune_train_loader)):
    if batch.edge_index.shape[0] == 0:
        edge_index = get_empty_edge_index(batch)
        batch.edge_index = edge_index
    input_ids = batch.input_ids
    batch_example_ct = batch.edge_index[1].shape[0]
    batch.pop("input_ids")
    attention_mask = batch.attention_mask
    batch.pop("attention_mask")
    graph_batch = batch
    example_ct += batch_example_ct

0it [00:00, ?it/s]/users/eleves-b/2020/thomas.li/.local/lib/python3.9/site-packages/torch_geometric/data/collate.py:177: UserWarning: An output with one or more elements was resized since it had shape [10], which does not match the required output shape [2, 10]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at ../aten/src/ATen/native/Resize.cpp:28.)
  value = torch.cat(values, dim=cat_dim or 0, out=out)
/users/eleves-b/2020/thomas.li/.local/lib/python3.9/site-packages/torch_geometric/data/collate.py:177: UserWarning: An output with one or more elements was resized since it had shape [478], which does not match the required output shape [2, 478]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse a

In [75]:
batch.edge_index.dtype

torch.int64

In [73]:
batch.x.shape

torch.Size([1073, 300])

In [84]:
import torch

tensor = torch.arange(33).unsqueeze(0)
concatenated_tensor = torch.cat((tensor, tensor), dim=0)
print(concatenated_tensor)

tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
         18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32],
        [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
         18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32]])


In [86]:
batch.x.shape[0]

1073

In [87]:
def get_empty_edge_index(batch):
    n_nodes = batch.x.shape[0]
    self_edge_index = torch.arange(n_nodes).unsqueeze(0)
    self_edge_index = torch.cat((self_edge_index, self_edge_index), dim=0)
    return self_edge_index

In [91]:
batch.edge_index.shape[0]

2

In [94]:
torch.tensor([]).shape[0]

0

In [88]:
if batch.edge_index.shape[0] == 0:
    edge_index = get_empty_edge_index(batch)
    batch.edge_index = edge_index

tensor([[   0,    1,    2,  ..., 1070, 1071, 1072],
        [   0,    1,    2,  ..., 1070, 1071, 1072]])

In [62]:
batch.ptr

tensor([   0,   56,  112,  134,  213,  284,  367,  375,  452,  523,  620,  708,
         716,  795,  880,  896,  977, 1054, 1133, 1208, 1224, 1246, 1303, 1356,
        1500, 1573, 1631, 1706, 1803, 1876, 1962, 2039, 2097])

In [69]:
batch.x.shape

torch.Size([33, 300])

In [68]:
batch.edge_index

tensor([], dtype=torch.int64)

In [ ]:
tensor

In [ ]:
fill_edge_index

In [70]:
batch.edge_index[1]

IndexError: index 1 is out of bounds for dimension 0 with size 0

In [22]:
finetune_train_loader, finetune_train_data_size = finetune_make(
    model,
    train_loader,
    criterion,
    example_ct_before=26000,
    config=finetune_config_instance,
)

  0%|          | 0/826 [00:00<?, ?it/s]/users/eleves-b/2020/thomas.li/.local/lib/python3.9/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 20 worker processes in total. Our suggested max number of worker in current system is 16, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
 21%|██        | 170/826 [00:30<01:43,  6.37it/s]/users/eleves-b/2020/thomas.li/.local/lib/python3.9/site-packages/torch_geometric/data/collate.py:177: UserWarning: An output with one or more elements was resized since it had shape [1960], which does not match the required output shape [2, 1960]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor 

{'len_miss_ranked': 14414, 'avg_miss_ranked': 5.262383793534064, 'max_miss_ranked': 704, 'total_miss_ranked_pairs': 90266}


/users/eleves-b/2020/thomas.li/.local/lib/python3.9/site-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [23]:
epochs = 1 + ((10 * 26000) // finetune_train_data_size)

In [28]:
example_ct = 0
for batch in tqdm(train_loader):
    input_ids = batch.input_ids
    batch_example_ct = batch.input_ids.shape[0]
    batch.pop("input_ids")
    attention_mask = batch.attention_mask
    batch.pop("attention_mask")
    graph_batch = batch
    example_ct += batch_example_ct

 29%|██▉       | 241/826 [00:06<00:11, 50.63it/s]/users/eleves-b/2020/thomas.li/.local/lib/python3.9/site-packages/torch_geometric/data/collate.py:177: UserWarning: An output with one or more elements was resized since it had shape [2188], which does not match the required output shape [2, 2188]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at ../aten/src/ATen/native/Resize.cpp:28.)
  value = torch.cat(values, dim=cat_dim or 0, out=out)
 36%|███▋      | 301/826 [00:07<00:11, 47.66it/s]/users/eleves-b/2020/thomas.li/.local/lib/python3.9/site-packages/torch_geometric/data/collate.py:177: UserWarning: An output with one or more elements was resized since it had shape [1982], which does not match the required output shape [2, 1982]. This behavior is deprecated, and in a future PyTorch release 

In [63]:
batch = next(iter(train_loader))

In [27]:
input_ids = batch.input_ids
batch_example_ct = batch.input_ids.shape[0]
batch.pop("input_ids")
attention_mask = batch.attention_mask
batch.pop("attention_mask")
graph_batch = batch

In [28]:
edge_index = torch.tensor(graph_batch.edge_index[:, :1000], dtype=torch.long)

/tmp/ipykernel_1299938/2286432983.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_index = torch.tensor(graph_batch.edge_index[:,:1000], dtype=torch.long)


## Model Pipeline

In [7]:
from sklearn.metrics.pairwise import cosine_similarity
from src.batch_sampler import get_batch_sampler
from src.finetune_evaluate import evaluate
from src.finetune_train import finetune_train_one_epoch
from src.finetune_make import finetune_make, make
from src.utils import get_date_time_string, LRAP_accuracy
import yaml


def finetune_model_pipeline(hyperparameters, model, model_path, example_ct_before):
    print(hyperparameters)
    model.load_state_dict(torch.load(model_path))  # a .pth file
    print("Loaded model")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    run_name = os.path.basename(model_path)[:-4] + "finetune" + get_date_time_string()
    print("run_name : ", run_name)

    # tell wandb to get started
    with wandb.init(project=PROJECT_NAME, config=hyperparameters, name=run_name):
        config = wandb.config

        tokenizer = AutoTokenizer.from_pretrained(config.model_name)
        gt = np.load("./data/token_embedding_dict.npy", allow_pickle=True)[()]
        train_dataset = GraphTextDataset(
            root="./data/", gt=gt, split="train", tokenizer=tokenizer
        )

        # make the model, data, and optimization problem
        print("Making dataloaders and loading model")
        train_loader, val_loader, criterion, optimizer = make(
            config, model, shuffle=False
        )

        finetune_train_loader, finetune_train_data_size = finetune_make(
            model, train_loader, criterion, example_ct_before, config
        )

        example_ct = example_ct_before
        val_similarity_matrix = evaluate(
            model,
            val_loader,
            criterion,
            cosine_similarity,
            LRAP_accuracy,
            "val",
            example_ct,
        )

        # and use them to train the model
        epochs = 1 + ((config.epoch_equivalents * 26400) // (finetune_train_data_size))
        print(f"Start training for {epochs} epochs on finetune dataset")
        example_ct = example_ct_before
        for epoch in tqdm(range(epochs)):
            finetune_train_one_epoch(
                model,
                finetune_train_loader,
                criterion,
                optimizer,
                example_ct_before=example_ct,
            )
            example_ct += finetune_train_data_size

            # train_similarity_matrix = evaluate(
            #     model,
            #     train_loader,
            #     criterion,
            #     cosine_similarity,
            #     LRAP_accuracy,
            #     "val",
            #     example_ct,
            # )

            val_similarity_matrix = evaluate(
                model,
                val_loader,
                criterion,
                cosine_similarity,
                LRAP_accuracy,
                "val",
                example_ct,
            )

            # Save the model at the end of each epoch
            folder_path = "models/" + run_name
            if not os.path.exists(folder_path):
                os.mkdir(folder_path)
            save_path = folder_path + "/" + run_name + "_" + str(epoch) + ".pth"
            torch.save(model.state_dict(), save_path)

            # # Get new finetune_train
            # print("Get new finetune data")
            # batch_sampler, stats_dico = get_batch_sampler(
            #     train_similarity_matrix, config.batch_size
            # )
            # print(stats_dico)
            # finetune_train_loader = DataLoader(
            #     train_dataset, batch_sampler=batch_sampler
            # )

        # Save the config
        yaml_file_path = folder_path + "/" + "config.yaml"
        with open(yaml_file_path, "w") as file:
            yaml.dump(wandb.config._as_dict(), file, default_flow_style=False)

        print("Successful training")

        return model

# VI) Run experiments

In [ ]:
finetune_config = dict(
    model_path="models/baseline_finetune/baseline_finetune.pth",
    model_name="distilbert-base-uncased",
    batch_size=32,
    graph_learning_rate=(3e-4) / 5,
    text_learning_rate=(3e-5) / 5,
    learning_rate=1e-4,
    epoch_equivalents=10,
    criterion="negative_sampling_contrastive_loss",
    similarity_fn="cosine_similarity",
)

finetune_model_pipeline(finetune_config, example_ct_before=2207904)

In [ ]:
finetune_model_pipeline(hyperparameters, model, model_path, example_ct_before)

In [8]:
# Imports
import wandb

wandb.login()
from src.Model4 import Model  # TODO
import torch


model_path = "models/michael_big_model_120_epochs_512_nhid_online_02021906/michael_big_model_120_epochs_512_nhid_online_02021906.pth"

# Config
finetune_config = dict(
    model_path=model_path,
    model_name="distilbert-base-uncased",
    batch_size=32,
    graph_learning_rate=(1e-4) / 10,
    text_learning_rate=(3e-5) / 10,
    learning_rate=1e-5,
    epoch_equivalents=10,
    criterion="negative_sampling_contrastive_loss",
    similarity_fn="cosine_similarity",
)

# Model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device :", device)

model = Model(
    model_name=finetune_config["model_name"],
    num_node_features=300,
    nout=768,
    nhid=512,
    graph_hidden_channels=512,
)
model.to(device)

# Run Name
run_name = "finetune_michael_big_model_120_epochs_512_nhid_online"


finetune_model_pipeline(finetune_config, model, model_path, example_ct_before=3168960)
# This block ensures that the code inside main() is only executed if the script is run directly
# if __name__ == "__main__":
#     finetune_model_pipeline(finetune_config, model, example_ct_before=3168960)

device : cuda
{'model_path': 'models/michael_big_model_120_epochs_512_nhid_online_02021906/michael_big_model_120_epochs_512_nhid_online_02021906.pth', 'model_name': 'distilbert-base-uncased', 'batch_size': 32, 'graph_learning_rate': 1e-05, 'text_learning_rate': 3e-06, 'learning_rate': 1e-05, 'epoch_equivalents': 10, 'criterion': 'negative_sampling_contrastive_loss', 'similarity_fn': 'cosine_similarity'}
Loaded model
run_name :  michael_big_model_120_epochs_512_nhid_online_02021906finetune_02031624


Making dataloaders and loading model


/users/eleves-b/2020/thomas.li/.local/lib/python3.9/site-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)
 22%|██▏       | 178/826 [00:15<00:51, 12.51it/s]/users/eleves-b/2020/thomas.li/.local/lib/python3.9/site-packages/torch_geometric/data/collate.py:177: UserWarning: An output with one or more elements was resized since it had shape [1960], which does not match the required output shape [2, 1960]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at ../aten/src/ATen/native/Resize.cpp:28.)
  value = torch.cat(values, dim=cat_dim or 0, out=out)
 36%|███▋      | 300/826 [00:24<00:42, 12.49it/s]/users/eleves-b/2020/thomas.li/.local/lib/python3.9/site-packages/torch_geometric/data/collate.py:177: UserWar

{'len_miss_ranked': 10874, 'avg_miss_ranked': 3.3677579547544605, 'max_miss_ranked': 479, 'total_miss_ranked_pairs': 47495}


/users/eleves-b/2020/thomas.li/.local/lib/python3.9/site-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


...

train_loss after 3486080 examples: 0.00230


100%|██████████| 4/4 [39:53<00:00, 598.32s/it]


Successful training


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train_current_loss,█▄▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_LRAP,█▁▃▄▅
val_loss,▁▇▆▇█
epoch,134.08
train_current_loss,0.0023
val_LRAP,0.59377
val_loss,0.00752


Model(
  (graph_encoder): GraphEncoder(
    (relu): ReLU()
    (ln): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (conv1): GCNConv(300, 512)
    (bn1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): GCNConv(512, 512)
    (bn2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv3): GCNConv(512, 512)
    (bn3): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (mol_hidden1): Linear(in_features=512, out_features=512, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (mol_hidden2): Linear(in_features=512, out_features=768, bias=True)
  )
  (text_encoder): TextEncoder(
    (bert): DistilBertModel(
      (embeddings): Embeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1,

In [7]:
# from src.Model4 import Model


# finetune_config = dict(
#     model_path="models/baseline_finetune/baseline_finetune.pth",
#     model_name="distilbert-base-uncased",
#     batch_size=32,
#     graph_learning_rate=(1e-4) / 10,
#     text_learning_rate=(3e-5) / 10,
#     learning_rate=1e-4,
#     epoch_equivalents=60,
#     criterion="negative_sampling_contrastive_loss",
#     similarity_fn="cosine_similarity",
# )

# finetune_model_pipeline(finetune_config, example_ct_before=2207904)